In [1]:
import os
from src.dataPipeline import DataPipeline
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10)

os.chdir('..')


columns_to_drop_all = [
'Unnamed: 0.1', 'Unnamed: 0','Municipality','Living space',
'Plot area','Floor space','location',
'description',
#'detailed_description',
'url','table','detail_responsive#municipality',
'detail_responsive#surface_living','detail_responsive#floor','detail_responsive#available_from',
'Gemeinde','Wohnfläche','Stockwerk','Nutzfläche','Verfügbarkeit','Grundstücksfläche',
'detail_responsive#surface_property','Commune','Surface habitable','Surface du terrain',
'Surface utile','Disponibilité','Étage','Comune','Superficie abitabile','Disponibilità',
'Piano','Superficie del terreno','Superficie utile','Municipality_merged',
'Floor_merged','Living_space_merged','Plot_area_merged','Availability_merged','location_parsed',
'title',
'price','address','link','details_structured','index',
'Locality','Zip','rooms','Floor_unified','Living_area_unified','space',
'price_s','address_s','Surface living:','Land area:',
'description_detailed','Floor space:','Volume:','plz','Number of toilets:','Gross yield:',
'Minimum floor space:','space_cleaned'

# Temporary columns
, 'Gross return'
, 'details'
, 'Room height:'
, 'features'
, 'type'
, 'provider' ,
'type_unified',
'Availability',
'Floor',
'detail_responsive#surface_usable',
'Floor_space_merged',
'lat',
'lon',
'ForestDensityL',
'ForestDensityM',
'ForestDensityS',
'Latitude',
'Longitude',
'NoisePollutionRailwayL',
'NoisePollutionRailwayM',
'NoisePollutionRailwayS',
'NoisePollutionRoadL',
'NoisePollutionRoadM',
'NoisePollutionRoadS',
'PopulationDensityL',
'PopulationDensityM',
'PopulationDensityS',
'RiversAndLakesL',
'RiversAndLakesM',
'RiversAndLakesS',
'WorkplaceDensityL',
'WorkplaceDensityM',
'WorkplaceDensityS',
'distanceToTrainStation',
'gde_area_agriculture_percentage',
'gde_area_forest_percentage',
'gde_area_nonproductive_percentage',
'gde_area_settlement_percentage',
'gde_average_house_hold',
'gde_empty_apartments',
'gde_foreigners_percentage',
'gde_new_homes_per_1000',
'gde_politics_bdp',
'gde_politics_cvp',
'gde_politics_evp',
'gde_politics_fdp',
'gde_politics_glp',
'gde_politics_gps',
'gde_politics_pda',
'gde_politics_rights',
'gde_politics_sp',
'gde_politics_svp',
'gde_pop_per_km2',
'gde_population',
'gde_private_apartments',
'gde_social_help_quota',
'gde_tax',
'gde_workers_sector1',
'gde_workers_sector2',
'gde_workers_sector3',
'gde_workers_total',
'plz_parsed',
'No. of rooms:',
'Number of apartments:',
'Last refurbishment:',
'Number of floors:',
'Year built:',
'Space extracted',
'Plot_area_unified',

]

dp = DataPipeline()
dp.readCsv("data/immo_data_202208_v2.csv")
dp.dropColumns(columns_to_drop_all)

df = dp.getData()
df

C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:42: DtypeWarning: Columns (3,4,5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50,107,110,114,115,116,119,120,121,124,125,126,128,131,132) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(filePath)


,detailed_description,price_cleaned
0,DescriptionLuxuriöse Attika-Wohnung direkt an ...,1150000.0
1,DescriptionStilvolle Liegenschaft an ruhiger L...,1420000.0
2,"detail_responsive#description_title2,5 Zimmerw...",720000.0
3,DescriptionDieses äusserst grosszügige Minergi...,1430000.0
4,DescriptionAus ehemals zwei Wohnungen wurde ei...,995000.0
...,...,...
22476,"Description\n""Hausteil mit verschieden Nutzung...",475000.0
22477,"Description\n""J'ADORE - Exklusives Wohnen in W...",1490000.0
22478,"Description\n""Einmalige Gelegenheit an sehr gu...",1450000.0
22479,"Description\n""LA VIE - Exklusives Wohnen in Ma...",1290000.0


In [3]:
# set the random seed to get comparable results
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

np.random.seed(0)

features = df['detailed_description']
# Fill NaN values with empty string
features = features.fillna('')
target = df['price_cleaned']

# Drop all columns with NaN values in price_cleaned
features = features[target.notnull()]
target = target[target.notnull()]

In [10]:

# prepare the data
comments_train, comments_test, y_train, y_test = train_test_split(features, target, test_size = 0.3) 

# we use a count vectorizer to break the individual comments into tokens, one token per word
vectorizer = CountVectorizer()
vectorizer.fit(comments_train)

# we create the training a matrix 
X_train = vectorizer.transform(comments_train)
X_test = vectorizer.transform(comments_test)

# train the classifier
#classifier = LogisticRegression(max_iter=2000)
#classifier.fit(X_train, y_train)
vectorizer.get_feature_names_out()

array(['00', '000', '0001', ..., 'œnothèque', 'œuvre', 'œuvres'],
      dtype=object)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

classifier = RandomForestRegressor()
classifier.fit(X_train, y_train)

In [ ]:
#test the data
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

#test the classifier with mape
from sklearn.metrics import mean_absolute_percentage_error
y_pred = classifier.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
print("MAPE:", mape)

In [ ]:
#plot the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

cm = confusion_matrix(y_test, classifier.predict(X_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

# Embedding for Text with BERT

In [6]:
from transformers import BertModel, BertTokenizer
import torch.nn as nn

class BertForRegression(nn.Module):
    def __init__(self, pretrained_model_name):
        super(BertForRegression, self).__init__()
        self.bert = BertModel.from_pretrained(pretrained_model_name)
        self.regressor = nn.Linear(self.bert.config.hidden_size, 1)
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.pooler_output  # `[CLS]` token output
        return self.regressor(cls_output)
    

In [7]:
from torch.utils.data import Dataset


class RealEstateDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        description = self.dataframe.iloc[index]['detailed_description']
        price = self.dataframe.iloc[index]['price_cleaned']

        # Tokenisierung
        encoding = self.tokenizer(
            description,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),  # Entferne Batch-Dimension
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'target': torch.tensor(price, dtype=torch.float)
        }

In [9]:
from torch.utils.data import DataLoader
from src import torchModelRun
import torch

# Train BERT model

model = BertForRegression('bert-base-uncased')

epochs = 10
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)
loss_fn = nn.MSELoss()

import pandas as pd
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

max_length = 128
train_dataset = RealEstateDataset(train_df, tokenizer, max_length)
test_dataset = RealEstateDataset(test_df, tokenizer, max_length)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)




#import sgd
from torch.optim import SGD
import torch.nn as nn

# Modell initialisieren
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForRegression('bert-base-uncased').to(device)

# Optimizer und Verlustfunktion
optimizer = SGD(model.parameters(), lr=1e-5)
loss_fn = nn.MSELoss()

# Training
epochs = 10

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target = batch['target'].view(-1, 1).to(device)
        
        optimizer.zero_grad()
        output = model(input_ids, attention_mask)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    print(f"Epoch {epoch+1}, Training Loss: {train_loss:.4f}")




Epoch 1, Training Loss: nan
Epoch 2, Training Loss: nan
Epoch 3, Training Loss: nan
Epoch 4, Training Loss: nan
Epoch 5, Training Loss: nan
Epoch 6, Training Loss: nan
Epoch 7, Training Loss: nan
Epoch 8, Training Loss: nan


KeyboardInterrupt: 